In [1]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Store.SQLite"
#r "nuget: Jinaga.Notebooks"
#r "nuget: Jinaga.UnitTest"

using Jinaga;
using Jinaga.Notebooks;
using Jinaga.Store.SQLite;
using Jinaga.UnitTest;

Installed Packages Jinaga, 1.1.33 Jinaga.Notebooks, 1.1.8 Jinaga.Store.SQLite, 1.1.10 Jinaga.UnitTest, 1.1.4

Loading extensions from `/Users/michaelperry/.nuget/packages/microsoft.data.analysis/0.22.0/interactive-extensions/dotnet/Microsoft.Data.Analysis.Interactive.dll`

# Jinaga Quick Start

Jinaga is a data management framework for mobile, web, and distributed applications.
It is particularly good for offline-first applications and progressive web apps, where data is stored locally and synchronized with a server.
We call that server a *replicator*, but we'll get to that.

## Jinaga Client

A Jinaga client manages data in your application and connects to a replicator.
Create a client using a factory method.
For example, to store local state in SQLite, use this method:

In [ ]:
JinagaClient j = JinagaSQLiteClient.Create(options =>
{
    // Connect to the Jinaga replicator that you host yourself, or that we host for you.
    options.HttpEndpoint = new Uri("https://rep.jinaga.com/myreplicator");

    // Use the SQLite store to persist data locally.
    options.SQLitePath = System.IO.Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.LocalApplicationData),
        "jinaga", "myapplication.sqlite"
    );
});

Or to use Jinaga in a unit test, use an in-memory store:

In [3]:
JinagaClient j = JinagaTest.Create(options =>
{
    options.User = new User("--- TEST USER ---");
});

## Facts

The core unit of data in Jinaga is a *fact*.
A fact is an immutable record of an event, decision, or state change.
Let's start simple.
When the user logs into your application, that's a fact.

In [4]:
(User user, UserProfile profile) = await j.Login();

j.RenderFacts(user)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== -->
 
 Gs+Fo0xO04hUAteaPwrHZDmyovTwr7asnKsBrkRf3HE3M9nYIj4Sk7ZhR8YK5uMq1SMHPrQohtQNwo9B7whK0w== 
 
 Jinaga.User 
 
 publicKey 
 
 --- TEST USER ---